In [15]:
import os
import pandas as pd
import numpy as np
import logging
import multiprocessing as mp

from commons import vizwiz as vw
from commons import images as im
from IPython.core.display import HTML

from tqdm import tqdm
tqdm.pandas()

# %pip install opencv-python-headless

In [2]:
vizwiz = vw.VizWiz(annotation_file='annotations/train.json')

loading annotations into memory...
Done (t=0.57s)
creating index...
index created! imgs = 23431, anns = 100575


In [20]:
vizwiz_df = pd.DataFrame.from_dict(vizwiz.dataset['annotations'], orient='columns')
vizwiz_images_df = pd.DataFrame.from_dict(vizwiz.dataset['images'], orient='columns')

vizwiz_df = vizwiz_df.merge(vizwiz_images_df.rename({'id': 'image_id', 'text_detected': 'image_text_detected'}, axis=1), 
                            on='image_id', how='left')

In [21]:
# word counts for each caption
vizwiz_df['words_count'] = vizwiz_df['caption'].str.split().apply(len)

# https://stackoverflow.com/questions/61299310/pandas-groupby-with-custom-function-to-return-the-column-values-as-an-array

In [16]:
%%time

# create df with unique image id and file name
cols = ['image_id', 'file_name']
vizwiz_unique = vizwiz_df.groupby(by=cols, as_index=False).first()[cols]

# find dimension for each image
bucket = 'assistive-vision'
train_imgs = 'vizwiz/train'

images = im.ImageS3(bucket)

def dimension(f):
    fpath = ''.join([train_imgs, '/', f])
    img = images.getImage(fpath)
    return img.shape

#vizwiz_unique['shape'] = vizwiz_unique['file_name'].progress_apply(lambda f: dimension(f))

# parallelize the tasks to all cpu cores
with mp.Pool(mp.cpu_count()) as pool:
    vizwiz_unique['shape'] = pool.map(dimension, vizwiz_unique['file_name'])

In [23]:

vizwiz_unique.drop(columns=['file_name'], inplace=True)
vizwiz_df = vizwiz_df.merge(vizwiz_unique, on='image_id', how='left')


In [28]:
display(HTML(vizwiz_df.head().to_html()))

,caption,image_id,is_precanned,is_rejected,id,text_detected,file_name,vizwiz_url,image_text_detected,words_count,shape
0,ITS IS A BASIL LEAVES CONTAINER ITS CONTAINS THE NET WEIGHT TOO.,0,False,False,0,True,VizWiz_train_00000000.jpg,https://ivc.ischool.utexas.edu/VizWiz_visualization_img/VizWiz_train_00000000.jpg,True,12,"(1632, 1224, 3)"
1,A green and white plastic condiment bottle containing Basil leaves.,0,False,False,1,True,VizWiz_train_00000000.jpg,https://ivc.ischool.utexas.edu/VizWiz_visualization_img/VizWiz_train_00000000.jpg,True,10,"(1632, 1224, 3)"
2,A bottle of spices in a plastic container laying on a surface.,0,False,False,3,True,VizWiz_train_00000000.jpg,https://ivc.ischool.utexas.edu/VizWiz_visualization_img/VizWiz_train_00000000.jpg,True,12,"(1632, 1224, 3)"
3,some basil leaves in a container on a counter,0,False,False,4,True,VizWiz_train_00000000.jpg,https://ivc.ischool.utexas.edu/VizWiz_visualization_img/VizWiz_train_00000000.jpg,True,9,"(1632, 1224, 3)"
4,"A can of Coca Cola on a counter is shown for when one can use a nice, cold drink.",1,False,False,5,True,VizWiz_train_00000001.jpg,https://ivc.ischool.utexas.edu/VizWiz_visualization_img/VizWiz_train_00000001.jpg,True,19,"(1296, 968, 3)"
